In [67]:
!pip install --upgrade pip

Consider using the `--user` option or check the permissions.


  Using cached pip-21.1.2-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1


In [69]:
!pip install torchvision==0.8.2 torchaudio==0.7.2

ERROR: Could not find a version that satisfies the requirement torchvision==0.8.2 (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.9.0, 0.9.1, 0.10.0)
ERROR: No matching distribution found for torchvision==0.8.2


In [73]:
!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT/
!pip install -r requirements.txt
!pip install .

urn self._get_build_requires(
    File "c:\users\源\x80\xec닔\xed쁽\appdata\local\programs\python\python39\lib\site-packages\setuptools\build_meta.py", line 127, in _get_build_requires
      self.run_setup()
    File "c:\users\源\x80\xec닔\xed쁽\appdata\local\programs\python\python39\lib\site-packages\setuptools\build_meta.py", line 248, in run_setup
      super(_BuildMetaLegacyBackend,
    File "c:\users\源\x80\xec닔\xed쁽\appdata\local\programs\python\python39\lib\site-packages\setuptools\build_meta.py", line 142, in run_setup
      exec(compile(code, __file__, 'exec'), locals())
    File "setup.py", line 2, in <module>
      from setuptools_rust import Binding, RustExtension
  ModuleNotFoundError: No module named 'setuptools_rust'
  ----------------------------------------
  ERROR: Command errored out with exit status 1:
   command: 'c:\users\김수현\appdata\local\programs\python\python39\python.exe' 'c:\users\김수현\appdata\local\programs\python\python39\lib\site-packages\pip\_vendor\pep517\in_pro

In [71]:
import torch
torch.__version__

'1.9.0+cpu'

In [34]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import get_linear_schedule_with_warmup

In [38]:
bertmodel, vocab = get_pytorch_kobert_model()

234

In [47]:
#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [52]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        print(self.labels, label_idx)
        self.labels.sort()
        print(self.labels)
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [29]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=25,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

,그 날을 잊지 못해 날 보며 환히 웃던 너의 미소에 홀린 듯 너무 쪽팔림에 난 그저 한마디 말도 못해 너의 눈빛은 날 자꾸 네 곁을 맴돌게 해 굳게 닫힌 내 맘이 어느새 무너져버려 온통 너의 생각뿐이야 나도 미치겠어 너무 보고 싶어 매일 매일 매일 자꾸 초라해지잖아 내 모습이 그대여 내게 말해줘 사랑한다고 하루가 멀다 하고 기다리고 있잖아 기다리고 있잖아 오늘 밤이 가기 전에 조금 더 다가와 줘 대체 뭘 고민해 빨리 안아 아닌 척 모르는 척 하다가 늦게 놓치고 후회 말아 너의 눈빛은 날 자꾸 네 곁을 맴돌게 해 굳게 닫힌 내 맘이 어느새 무너져버려 온통 너의 생각뿐이야 나도 미치겠어 너무 보고 싶어 매일 매일 매일 자꾸 초라해지잖아 내 모습이 그대여 내게 말해줘 사랑한다고 하루가 멀다 하고 기다리고 있잖아 이제 와 숨기려 하지 마요 그대여 아닌 척하지 마요 온종일 난 그대 생각에 잠긴 채로 난 이대로 기다리고 있어요 하루가 멀다 하고 기다리고 있잖아 기다리고 있잖아,2
0,나리는 꽃가루에 눈이 따끔해 아야 눈물이 고여도 꾹 참을래 내 마 한켠 비밀스런 오...,0
1,넌 운전만 해 계속 운전만 해 왜 이리 된 걸까 우리 사이가 갑자기 어색해졌단 걸 ...,5
2,세상의 모서리 구부정하게 커버린 골칫거리 걸음걸이 옷차림 이어폰 너머 악까지 다 넌...,1
3,어쩜 살아가다 보면 한 번은 날 찾을지 몰라 나 그 기대 하나로 오늘도 힘겹게 버틴...,15
4,그대는 참 아름다워요 밤 하늘의 별빛보다 빛나요 지친 나의 마음을 따뜻하게 감싸줄 ...,21
...,...,...
85,희망을 걸어 모두가 날더러 더 힘들 거라고 얘기했어 그래서 뭐 난 너를 믿어 추운 ...,1
86,그 날을 잊지 못해 날 보며 환히 웃던 너의 미소에 홀린 듯 너무 쪽팔림에 난 그저...,7
87,나리는 꽃가루에 눈이 따끔해 아야 눈물이 고여도 꾹 참을래 내 마 한켠 비밀스런 오...,1
88,넌 운전만 해 계속 운전만 해 왜 이리 된 걸까 우리 사이가 갑자기 어색해졌단 걸 ...,11


In [56]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5

In [58]:
import pandas as pd
df = pd.read_csv('/content/sample_data/result.csv', names=['lyrics', 'mood'])
df.to_csv('/content/sample_data/result_nonheader.csv', index=False, header=False)
df = pd.read_csv('/content/sample_data/result_nonheader.csv', header=None)


In [60]:
data = [[df.iloc[i][0], df.iloc[i][1]] for i in range(len(df))]

In [61]:
device = torch.device("cpu")


Global seed set to 42
Using PyTorch Ver 1.9.0+cpu
Fix Seed: 42
Downloading: 100%|██████████| 672/672 [00:00<00:00, 343kB/s]
Downloading: 100%|██████████| 1.34G/1.34G [04:48<00:00, 4.66MB/s]
Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you 

MisconfigurationException: You have asked for native AMP on CPU, but AMP is only available on GPU.

In [ ]:
# train, validation, test set을 나눠주세요
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data, test_size=0.1, random_state=123)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
#모델을 만들고 GPU 사용 설정을 해줍니다
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
#옵티마이저와 손실함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_linear_schedule_with_warmup(optimizer, warmup_step, t_total)

#정확도를 계산하기 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#학습 과정
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad() 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        label = label.long().to(device) 
        out = model(token_ids, valid_length, segment_ids) 
        loss = loss_fn(out, label) 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step() 
        scheduler.step()  # Update learning rate schedule 
        train_acc += calc_accuracy(out, label)  
        if batch_id % log_interval == 0: 
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1))) 
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1))) 
    model.eval() #모델 평가 부분 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
##################################################
# 평가용 Test_set을 모델에 입력하기 위해 형식을 맞춰줍니다

sentence_eval["emotion"] = sentence_eval["emotion"].apply(label)

dtls_eval = [list(sentence_eval.iloc[i,:]) for i in range(len(sentence_eval))]

data_test = BERTDataset(dtls_eval, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
# 해당 데이터에 대해 분류를 시작합니다
model.eval()
answer=[]
train_acc = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  max_vals, max_indices = torch.max(out, 1)
  answer.append(max_indices.cpu().clone().numpy())
  test_acc += calc_accuracy(out, label)
print('정답률: ',test_acc / (batch_id+1))

In [ ]:
# 제출 형식에 맞춰 파일을 저장해줍니다
ls = []
for i in answer:
  ls.extend(i)

pred = pd.DataFrame(ls, columns=['Predicted'])
df = pd.concat([sentence_eval['raw_text'], pred['Predicted'], sentence_eval['emotion']], axis=1) 

def test(x):
  if x==0.0: return '슬픔'
  elif x==1.0: return '기쁨'
  elif x==2.0: return '분노'
  elif x==3.0: return '공포'
  else: return x

df["Predicted"] = df["Predicted"].apply(test)
df["emotion"] = df["emotion"].apply(test)

for i in ['슬픔','기쁨','분노','공포']: 
  print(i, '개수', len(df[df['emotion'] == i]))
  print('예측 개수', len(df[df['emotion'] == i][df['Predicted'] == i]))
  print('정답률',len(df[df['emotion'] == i][df['Predicted'] == i])/len(df[df['emotion'] == i]))